In [1]:
import networkx as nx
import wikipediaapi
import graphviz

In [2]:
user_agent = 'wikipedia-graph (danielwarkus@hotmail.com)'

In [3]:
wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language='de')

In [4]:
bienen_page = wiki.page('Bienen')

In [5]:
def get_backlinks(page):
    backlinks = []
    for link in page.backlinks.values():
        backlinks.append(link.title)
    return backlinks

In [6]:
def get_links(page):
    links = []
    for link in page.links.values():
        links.append(link.title)
    return links

In [7]:
def get_page(link):
    return wiki.page(link)

In [8]:
def get_graph(page, depth):
    graph = nx.DiGraph()
    graph.add_node(page.title)
    links = get_links(page)
    for link in links:
        graph.add_edge(page.title, link)
        if depth > 0:
            next_page = get_page(link)
            graph = nx.compose(graph, get_graph(next_page, depth-1))
    return graph

In [9]:
def get_backgraph(page, depth):
    backgraph = nx.DiGraph()
    backgraph.add_node(page.title)
    backlinks = get_backlinks(page)
    for link in backlinks:
        backgraph.add_edge(page.title, link)
        if depth > 0:
            next_page = get_page(link)
            backgraph = nx.compose(backgraph, get_graph(next_page, depth-1))
    return backgraph

# Erstelle den Ursprungsgraphen
### Hier wird ein Graph erstellt, der alle Links (und deren Links) von der Bienen-Seite enthält.
### Die Artikelnamen sind dabei die Knoten und die Verlinkungen die Kanten. 

In [11]:
graph = get_graph(bienen_page, 1)

In [14]:
backlinks = get_backlinks(bienen_page)

In [15]:
print(backlinks)

['Erbse', 'Elefanten', 'Geist', 'Gewürzvanille', 'Heidekrautgewächse', 'Kürbisgewächse', 'Krieg', 'Lebewesen', 'Mimikry', 'Orchideen', 'Rosengewächse', 'Speierling', 'Salbei', 'Tropen', 'Kakteengewächse', 'Insekten', 'Färöer', 'Jahreszeit', 'Männliches Geschlecht', 'Schwanenblume', 'Weibliches Geschlecht', 'Ultraviolettstrahlung', 'Westliche Honigbiene', 'Akazien', 'Magnetit', 'Wikipedia:Humorarchiv/Leuchtschnabelbeutelschabe', 'Huflattich', 'Subtropen', 'Insektizid', 'Gewöhnliches Hirtentäschel', 'Weiden (Gattung)', 'Nachtschatten', 'Insektenkunde', 'Felsberg (Hessen)', 'Neuntöter', 'Hautflügler', 'Rambutan', 'Bienenkönigin', 'Ameisensäure', 'Schachblume', 'Gonosom', 'R- und S-Sätze', 'Acetylcholin', 'Ambrosius von Mailand', 'Brombeeren', 'Kölner Dom', 'Himbeere', 'Hummeln', 'El Hierro', 'Nest', 'Arbeitsteilung', 'Diskussion:Arbeitsteilung', 'Feuerbrand', 'Gänseblümchen', 'Liste der Schutzpatrone', 'Barfüßigkeit', 'Atrazin', 'Gemeiner Efeu', 'Pseudoskorpione', 'Gurke', 'Gelb', 'Erdmag

In [16]:
backgraph = get_backgraph(bienen_page, 1)

In [17]:
complete_graph = nx.compose(graph, backgraph)

In [12]:
# from graph, export edges as a tuple to a csv file
edges = graph.edges
with open('edges_original.csv', 'w', encoding='utf-8') as f:
    for edge in edges:
        f.write(f'{edge[0]},{edge[1]}\n')

# Versuchserklärung
#### Ziel ist es, Artikelvorschläge zu generieren, die thematisch zu Bienen passen. Dabei werde ich 4 verschiedene Ansätze testen (dabei wird immer der höchste Grad der Knoten betrachtet):
#### 1. Extrahiere die Knoten mit dem höchsten Grad (ungefiltert)
#### 2. Betrachte nur die direkten Nachfolger von Bienen
#### 3. Betrachte nur die Knoten, die zu Bienen verlinkt sind
#### 4. Betrachte nur die direkten Nachfolger von Bienen, die auch eine Verbindung zurück zu Bienen haben (beide Bedingungen)

#### Die Tests 2-4 wurde ich sowohl am Hauptgraphen, als auch an einem Subgraphen durchführen, um zu schauen ob dies ein Unterschied macht.

# Versuch 1: Extrahiere die Knoten mit dem höchsten Grad
#### Im ersten Versuch werden die Knoten mit dem höchsten Grad extrahiert. Man kann erkennen, dass die Artikelvorschläge zu oberflächlich sind. 

In [13]:
# print the top 3 nodes with the highest degree
top_3_nodes = sorted(graph.degree, key=lambda x: x[1], reverse=True)[:3]
print(top_3_nodes)

[('Deutschland', 2368), ('New York City', 871), ('Bernstein', 713)]


# Versuch 2: Extrahiere die direkten Nachfolger von Bienen
#### Um die Auswahl passender Artikel einzuschränken, werden nur die direkten Nachfolger von Bienen betrachtet.
#### Diese neue Bedingung soll dazu führen, nur Artikel in Betracht zu ziehen, die auch von der Biene refenziert werden.

#### Getestet habe ich diese Bedinung an zwei Graphen:
#### 1. Der ursprüngliche Graph
#### 2. Ein Subgraph, der nur die Knoten enthält, die von Bienen referenziert werden.
 

In [14]:
links_bienen = get_links(bienen_page)

In [15]:
reduced_node_list = [node for node in graph.nodes() if node in links_bienen]

## Erstelle Subgraphen und extrahiere die Knoten mit dem höchsten Grad
#### Hier wird ein Subgraph erstellt, der nur die Knoten enthält, die direkte Nachfolger von Bienen sind.
#### Die Ergebnisse sind besser, als die des ursprünglichen Graphen.

In [16]:
# create a subgraph
subgraph = graph.subgraph(reduced_node_list)

In [17]:
top_3_reduced_nodes = sorted(subgraph.degree(), key=lambda x: x[1], reverse=True)[:3]
print(top_3_reduced_nodes)

[('Systematik (Biologie)', 92), ('Gemeinsame Normdatei', 89), ('Nomenklatur (Biologie)', 77)]


## Extrahiere Knoten mit dem höchsten Grad aus Hauptgraphen
#### Hier zeigt die Zusatzbedingung keine Verbesserung.

In [18]:
top_3_reduced_nodes = sorted(graph.degree(reduced_node_list), key=lambda x: x[1], reverse=True)[:3]
print(top_3_reduced_nodes)

[('Deutschland', 2368), ('New York City', 871), ('Bernstein', 713)]


## Zusammenfassung Versuch 2
#### Man kann sehen, dass die reine Bedingung an den Knoten des Hauptgraphen noch nicht aussreicht für gute Ergebnisse. -> Zu oberflächliche Themen. Der Subgraph hingegen, schlägt zu spezifische Artikel vor.  

In [19]:
# export the subgraph to a csv file
edges = subgraph.edges
with open('edges_reduced.csv', 'w', encoding='utf-8') as f:
    for edge in edges:
        f.write(f'{edge[0]},{edge[1]}\n')

# Versuch 3: Extrahiere Knoten, die zu Bienen verlinkt sind
#### Hier wird die Bedingung getestet, dass nur die Knoten betrachtet werden die zu Bienen verlinkt sind.

#### Ebenfalls werde ich dies hier am Hauptgraphen und an einem Subgraphen testen.

In [20]:
nodes_with_edge_to_bienen = [node for node in graph.nodes() if graph.has_edge(node, 'Bienen')]

## Mit Ursprungsgaphen
#### Es zeigt sich, dass wir ein durchwegs gute Themenvorschläge erhalten.

In [21]:
top_3_linked = sorted(graph.degree(nodes_with_edge_to_bienen), key=lambda x: x[1], reverse=True)[:3]
print(top_3_linked)

[('Insekten', 492), ('Holzbienen', 464), ('Tropen', 308)]


## Mit Subgraphen
#### Deutlich erkennt man, dass die Themenvorschläge zu spezifisch sind.

In [29]:
subgraph_linked = graph.subgraph(nodes_with_edge_to_bienen)
top_3_linked = sorted(subgraph_linked.degree(), key=lambda x: x[1], reverse=True)[:5]
print(top_3_linked)

[('Apoidea', 80), ('Stechimmen', 73), ('Echte Bienen', 59), ('Taillenwespen', 46), ('Hummeln', 36)]


In [23]:
# export the subgraph to a csv file
edges = subgraph_linked.edges
with open('edges_linked.csv', 'w', encoding='utf-8') as f:
    for edge in edges:
        f.write(f'{edge[0]},{edge[1]}\n')

## Zusammenfassung Versuch 3
#### Diese Bedingung sorgt für signifikant bessere Vorschläge am Hauptgraphen. Der Subgraph hingegen, schlägt zu spezifische Artikel vor.

# Versuch 4: Extrahiere Knoten mit beiden Bedingungen
#### Zum Schluss setzten wir beide Bedingungen vorraus. Auch hier wird dies am Hauptgraphen und an einem Subgraphen getestet.

In [24]:
nodes_with_edge_to_bienen_reduced = [node for node in reduced_node_list if graph.has_edge(node, 'Bienen')]
print(nodes_with_edge_to_bienen_reduced)

['Ackerwinde', 'Insekten', 'Spiralhornbienen', 'Anthophora', 'Apoidea', 'Schmuckbienen', 'Stechimmen', 'Taillenwespen', 'Glanzbienen', 'Halictidae', 'Wildbiene', 'Andrenidae', 'Colletidae', 'Grabwespen', 'Hautflügler', 'Melittidae', 'Stenotritidae', 'Tropen', 'Anthidium', 'Große Wollbiene', 'Fleckenbienen', 'Kegelbienen', 'Trauerbienen', 'Westliche Honigbiene', 'Anthophoridae', 'Holzbienen', 'Apinae', 'Eucera', 'Honigbienen', 'Hummeln', 'Kraftbienen', 'Kuckuckshummeln', 'Kurzhornbienen', 'Körbchensammler', 'Melipona', 'Prachtbienen', 'Stachellose Bienen', 'Tetralonia', 'Triepeolus', 'Trigona (Gattung)', 'Crabronidae', 'Imker', 'Arbeitsteilung', 'Subtropen', 'Biene (Begriffsklärung)', 'Bienen (Begriffsklärung)', 'Bienenkönigin', 'Giftstachel', 'Weltbienentag', 'Blutbienen', 'Halictus', 'Lasioglossum', 'Brut', 'Nest', 'Sandbienen', 'Buntbienen', 'Zottelbienen', 'Charles Duncan Michener', 'Osmia', 'Maskenbienen', 'Seidenbienen', 'Kaltgemäßigte Klimazone', 'Kühlgemäßigte Klimazone', 'Dioxy

## Extraktion mit beiden Bedingungen am Hauptgraphen
#### Hier wird die Kombination von beiden Bedingungen am Hauptgraphen getestet. Diese Erbnisse unterschieden sich jedoch nicht von den Ergebnissen des dritten Versuchs.

In [25]:
top_3_clean = sorted(graph.degree(nodes_with_edge_to_bienen_reduced), key=lambda x: x[1], reverse=True)[:3]
print(top_3_clean)

[('Insekten', 492), ('Holzbienen', 464), ('Tropen', 308)]


## Erstellung und Extraktion mit beiden Bedingungen an eines Subgraphen
#### Dieser Subgraph enthält nur die Knoten, die beide Bedingungen.
#### Hier sind die ebenso wie in Versuch 3 die Themenvorschläge identisch. Das Einzige worin sich die Ergebnisse unterscheiden, ist eine abnahme im Degree.

In [26]:
subgraph_clean = graph.subgraph(nodes_with_edge_to_bienen_reduced)

In [27]:
top_3_clean = sorted(subgraph_clean.degree(nodes_with_edge_to_bienen_reduced), key=lambda x: x[1], reverse=True)[:5]
print(top_3_clean)

[('Apoidea', 66), ('Stechimmen', 59), ('Taillenwespen', 35), ('Hummeln', 32), ('Halictidae', 27)]


In [28]:
# export the subgraph to a csv file
edges = subgraph_clean.edges
with open('edges_predecessors.csv', 'w', encoding='utf-8') as f:
    for edge in edges:
        f.write(f'{edge[0]},{edge[1]}\n')

## Zusammenfassung Versuch 4:
#### Auch hier schneidet der Hauptgraph besser ab als der Subgraph.    

# Fazit
#### Die besten Ergebnissen wurden am Hauptgraphen mit der Bedingung erzielt, dass nur die Knoten betrachtet werden, die zu Bienen verlinkt sind. Dass beide Bedungungen erfüllt sein müssen, hat keinen Unterschied gemacht.
#### Der Subgraph hat bei allen Versuchen zu spezifische Artikel vorgeschlagen. Dies könnte man damit erklären, dass Informationen vom Hauptgraphen verloren gehen, da nur ein Teil der Knoten betrachtet wird. Dadurch fehlen Ingoing und Outgoing Links, die den Degree beeinflussen.   

In [31]:
backlinks = bienen_page.backlinks


In [33]:
top_x_backlinks = sorted(backlinks, key=lambda x: x[1], reverse=True)[:5]
print(top_x_backlinks)

['O’Neill-Zylinder', 'Kürbisgewächse', 'Nützling', 'Rübsen (Pflanze)', 'Würzburger Lügensteine']


In [19]:
pr_result = nx.pagerank(complete_graph)

In [20]:
top_n_pr = sorted(pr_result, key=lambda x: pr_result[x], reverse=True)[:5]

In [21]:
print(top_n_pr)

['Bienen', 'Systematik (Biologie)', 'Familie (Biologie)', 'Nomenklatur (Biologie)', 'Gattung (Biologie)']
